In [1]:
# IMPORT DES DONNEES
import pandas as pd 
import os
import time

# Import Selenium
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains

In [19]:
# set up the driver
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"  # specify the path to your chrome binary location

# Get the driver path
driver_path = ChromeDriverManager().install()

# Set ChromeDriver path
os.environ["PATH"] += os.pathsep + os.path.dirname(driver_path)

url = "https://www.payscale.com/research/US/Job=Software_Engineer/Salary"
url_base = "https://www.payscale.com/research/US/Job/Information-Technology/A"
driver = webdriver.Chrome(options=options)
driver.get(url)

In [20]:
# Splitting the URL at the '=' symbol and taking the second part
# Then split again at the '/' symbol to isolate the job title and replace underscores with spaces
job_title = url.split('=')[1].split('/')[0].replace('_', ' ')

In [21]:
job_title

'Software Engineer'

In [26]:
# Remove the overlay if it exists
driver.execute_script("""
if (document.querySelector('.onetrust-pc-dark-filter')) {
    document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';
}
""")

# Wait for the element to be clickable
read_more_button_1 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '(//span[@class="expandable__more"])[1]'))
)

# Scroll to the element
driver.execute_script("arguments[0].scrollIntoView();", read_more_button_1)

# Adjust the viewport
driver.execute_script("window.scrollBy(0, -100);")  # adjusted more to avoid potential obstructions

# Remove the overlay
driver.execute_script("document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';")

# Add an extended delay
time.sleep(5)

# Try to click using WebDriver with a retry logic
for _ in range(3):  # tries to click 3 times
    try:
        actions = ActionChains(driver)
        actions.move_to_element(read_more_button_1).click(read_more_button_1).perform()
        break
    except ElementClickInterceptedException:
        time.sleep(1)
else:
    driver.execute_script("arguments[0].click();", read_more_button_1)

# Fetch the expanded text for salary_by_exp
salary_by_exp_element = read_more_button_1.find_element(By.XPATH, '..')
salary_by_exp = salary_by_exp_element.text

In [27]:
if salary_by_exp.endswith("Read less"):
    salary_by_exp = salary_by_exp[:-9].strip()  # 9 is the length of "Read less", and we strip to remove any potential whitespace

In [28]:
salary_by_exp

'An entry-level Software Engineer with less than 1 year experience can expect to earn an average total compensation (includes tips, bonus, and overtime pay) of $80,131 based on 6,258 salaries. An early career Software Engineer with 1-4 years of experience earns an average total compensation of $89,324 based on 35,983 salaries. A mid-career Software Engineer with 5-9 years of experience earns an average total compensation of $101,761 based on 12,844 salaries. An experienced Software Engineer with 10-19 years of experience earns an average total compensation of $113,840 based on 4,184 salaries. In their late career (20 years and higher), employees earn an average total compensation of $120,621.'

In [22]:
# Remove the overlay if it exists
driver.execute_script("""
if (document.querySelector('.onetrust-pc-dark-filter')) {
    document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';
}
""")

# Attendez que le bouton "Read more" soit cliquable
read_more_button_2 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '(//span[@class="expandable__more"])[2]'))
)

# Cliquer sur le bouton "Read more" en utilisant JavaScript
driver.execute_script("arguments[0].click();", read_more_button_2)

# Augmentez le temps d'attente pour s'assurer que le contenu est chargé
time.sleep(10)  # Vous pouvez ajuster cette valeur en fonction de la rapidité de chargement de la page

# Récupérer chaque paragraphe
# Step 1: Localisez le conteneur parent
parent_container_xpath = '/html/body/div[1]/div/div[3]/div/div[1]/div/div'
parent_container = driver.find_element(By.XPATH, parent_container_xpath)

# Step 2: Récupérez tous les éléments <p>
all_paragraphs = parent_container.find_elements(By.TAG_NAME, "p")

# Step 3: Ajoutez le texte de chaque élément <p> à une liste
paragraphs = [para.text for para in all_paragraphs if para.text.strip() != ""]
    
job_desc = " ".join(paragraphs)

In [23]:
# print(driver.title)
# print(driver.current_url)
# driver.save_screenshot('screenshot.png')

In [24]:
if job_desc.endswith("Read less"):
    job_desc = job_desc[:-9].strip()  # 9 is the length of "Read less", and we strip to remove any potential whitespace

In [25]:
job_desc

'In the world of software design and development, a software engineer plays a key role. The engineer is typically the person who helps to develop the ways that software functions created by a software design team will work. The software engineer will work with designers to help consolidate disparate program functions into a unified whole. The engineer also works with programmers and coders to help map out various programming tasks and smaller functions, which are then combined into larger, functioning programs or new features for existing software. The engineer will typically work in both design and development stages of the software creation. During the design phase, dedicated designers or design teams will work with the engineer to help work out the basic things the program or update will be expected to perform. The engineer will typically help to plot out the various aspects of the automated tasks that will be necessary, usually using design documentation and flowcharts to help illu

In [9]:
def scrape_payscale_jobs(base_url, start_letter='A', end_letter='W'):
    driver = webdriver.Chrome()  # Make sure to have the correct options if needed
    all_data = []

    for letter in range(ord(start_letter), ord(end_letter)+1):
        print(f"Page {chr(letter)} en cours d'extraction")
        current_url = base_url + chr(letter)
        driver.get(current_url)

        job_links = driver.find_elements(By.CSS_SELECTOR, ".subcats__links__item")  # Fetch job links on the current page
        job_urls = [job_link.get_attribute("href") for job_link in job_links]

        for job_url in job_urls:
            driver.get(job_url)

            # Splitting the URL at the '=' symbol and taking the second part
            # Then split again at the '/' symbol to isolate the job title and replace underscores with spaces
            job_title = job_url.split('=')[1].split('/')[0].replace('_', ' ')
            print(job_title)

            # Remove the overlay if it exists
            driver.execute_script("""
            if (document.querySelector('.onetrust-pc-dark-filter')) {
                document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';
            }
            """)
            
            # Wait for the element to be clickable
            read_more_button_1 = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '(//span[@class="expandable__more"])[1]'))
            )
            
            # Scroll to the element
            driver.execute_script("arguments[0].scrollIntoView();", read_more_button_1)
            
            # Adjust the viewport
            driver.execute_script("window.scrollBy(0, -100);")  # adjusted more to avoid potential obstructions
            
            # Remove the overlay
            driver.execute_script("document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';")
            
            # Add an extended delay
            time.sleep(5)
            
            # Try to click using WebDriver with a retry logic
            for _ in range(3):  # tries to click 3 times
                try:
                    actions = ActionChains(driver)
                    actions.move_to_element(read_more_button_1).click(read_more_button_1).perform()
                    break
                except ElementClickInterceptedException:
                    time.sleep(1)
            else:
                driver.execute_script("arguments[0].click();", read_more_button_1)
            
            # Fetch the expanded text for salary_by_exp
            salary_by_exp_element = read_more_button_1.find_element(By.XPATH, '..')
            salary_by_exp = salary_by_exp_element.text

            
            # Remove the overlay if it exists
            driver.execute_script("""
            if (document.querySelector('.onetrust-pc-dark-filter')) {
                document.querySelector('.onetrust-pc-dark-filter').style.display = 'none';
            }
            """)
            
            # Attendez que le bouton "Read more" soit cliquable
            read_more_button_2 = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '(//span[@class="expandable__more"])[2]'))
            )
            
            # Cliquer sur le bouton "Read more" en utilisant JavaScript
            driver.execute_script("arguments[0].click();", read_more_button_2)
            
            # Augmentez le temps d'attente pour s'assurer que le contenu est chargé
            time.sleep(10)  # Vous pouvez ajuster cette valeur en fonction de la rapidité de chargement de la page
            
            # Récupérer chaque paragraphe
            # Step 1: Localisez le conteneur parent
            parent_container_xpath = '/html/body/div[1]/div/div[3]/div/div[1]/div/div'
            parent_container = driver.find_element(By.XPATH, parent_container_xpath)
            
            # Step 2: Récupérez tous les éléments <p>
            all_paragraphs = parent_container.find_elements(By.TAG_NAME, "p")
            
            # Step 3: Ajoutez le texte de chaque élément <p> à une liste
            paragraphs = [para.text for para in all_paragraphs if para.text.strip() != ""]
                
            job_desc = " ".join(paragraphs)

            # Stockez les données dans un dictionnaire, par exemple:
            job_data = {
                "title": job_title,
                "salary_by_exp": salary_by_exp,
                "description": job_desc
            }
            all_data.append(job_data)

            driver.back()  # Revenir à la liste des jobs pour la lettre en cours

    driver.quit()
    return all_data

base_url = "https://www.payscale.com/research/US/Job/Information-Technology/"

In [ ]:
data = scrape_payscale_jobs(base_url)

Page A en cours d'extraction
ABAP Developer
ASIC Design Engineer
ASIC Engineer
ASP.NET Developer


In [ ]:
job_desc